In [1]:
from sys import platform
import os
import cv2
import json
from pupil_apriltags import Detector
import open3d as o3d
from glob import glob
import numpy as np
from scipy.spatial.transform import Rotation as R
import collections
import k3d
import math

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:

def position_error(T_e, T_gt):

    return np.linalg.norm(T_e- T_gt)


def rotation_error(R_e, R_gt):
    
    return (180/math.pi) * np.arccos((np.trace(np.matmul(R_e, R_gt.T))-1)/2)

In [5]:
def k3d_frustrum(pose, name, size=0.009, color=0x0000ff):
    # i.e. not using image sizes 
    pos = pose[:3, 3]
    
    forward = -pose[:3, 2] * size * -1.4
    right = -pose[:3, 0] * size * 1.25
    up = -pose[:3, 1] * size
    
    #verts = [pos, pos + forward*size ]
    verts = [pos, pos + forward - right*0.5 + up*0.5, pos + forward + right * 0.5 + up * 0.5, pos ]
    verts += [pos, pos + forward - right*0.5 - up*0.5, pos + forward + right * 0.5 - up * 0.5, pos ]
    verts += [pos, pos + forward - right*0.5 + up*0.5, pos + forward - right * 0.5 - up * 0.5, pos]
    verts += [pos, pos + forward + right*0.5 + up*0.5, pos + forward + right * 0.5 - up * 0.5, pos]
    return k3d.line(verts, color=color, width=2.5, name = name, shader="simple")

def visualization_registration(reg, mesh):
    vertices = [ ]

    frustrums = []

    colors = [0xff0000, 0x00ff00, 0x0000ff, 0xffcc00, 0xccff00, 0x00ccff]

    for i, frame in enumerate(reg.keys()):
        pos = reg[frame][:3, 3] 
        vertices += pos.tolist()
        frustrums.append( k3d_frustrum(reg[frame], name = str(frame), size=0.1, color=colors[i % len(colors)]) )

    vertices = np.array(vertices)

    lines = k3d.line(vertices, color=0xff0000, width=2.5, shader="simple") # + line
    pts = k3d.points(vertices, point_size=0.003)

    plot3d = k3d.plot(antialias=True, camera_auto_fit=True)

    #plot3d += lines
    plot3d += pts 

    for f in frustrums:
        plot3d += f

    plot3d += k3d.points( [vertices[:3]], point_size=0.0075, color=0x00ff00)

    plot3d += k3d.mesh(np.array(mesh.vertices), np.array(mesh.triangles).flatten(), color=0xffcc00)
    plot3d.display()

def linear_fit(x, y, err = "Mean"):
    """For set of points `(xi, yi)`, return linear polynomial `f(x) = k*x + m` that
    minimizes the sum of quadratic errors.
    """
    meanx = sum(x) / len(x)
    meany = sum(y) / len(y)
    k = sum((xi-meanx)*(yi-meany) for xi,yi in zip(x,y)) / sum((xi-meanx)**2 for xi in x)
    m = meany - k*meanx

    sum_error = 0
    for xi, yi in zip(x,y):
        y_line = k*xi + m
        sum_error += abs(y_line - yi)
    
    if err == "Mean":
        error = sum_error / len(x)
    elif err == "Sum":
        error = sum_error

    return k, m, error

def visualization_EST_GT_IPhone(gt, reg,  mesh):
    vertices = [ ]

    frustrums = []

    colors = [0xff0000, 0x00ff00]

    for frame in reg.keys():
        est_pose = reg[frame][:3, 3]
        gt_pose = gt[frame][:3, 3]
 
        vertices += est_pose.tolist()
        vertices += gt_pose.tolist()
        frustrums.append( k3d_frustrum(reg[frame], name = str(frame) + "_GT", size=0.1, color=colors[0]) )
        frustrums.append( k3d_frustrum(gt[frame], name = str(frame)+ "Est", size=0.1, color=colors[1]) )

    vertices = np.array(vertices)

    lines = k3d.line(vertices, color=0xff0000, width=2.5, shader="simple") # + line
    pts = k3d.points(vertices, point_size=0.003)

    plot3d = k3d.plot(antialias=True, camera_auto_fit=True)

    #plot3d += lines
    plot3d += pts 

    for f in frustrums:
        plot3d += f

    plot3d += k3d.points( [vertices[:3]], point_size=0.0075, color=0x00ff00)

    plot3d += k3d.mesh(np.array(mesh.vertices), np.array(mesh.triangles).flatten(), color=0xffcc00)
    plot3d.display()

at_detector = Detector(
            families="tagStandard41h12",
            nthreads=1,
            quad_decimate=1.0,
            quad_sigma=0.0,
            refine_edges=1,
            decode_sharpening=0.25,
            debug=0,
        )


Register = collections.namedtuple(
    "RegisterInfo", ["CameraPose", "Intrinsic", "TagPose"])


In [19]:
if platform == "linux" or platform == "linux2":  
    # linux
    data_path  = "/home/biyang/Documents/3D_Gaze/dataset/3D_scanner_app/Apriltag1_dataset1"
    data_pi_path = "/home/biyang/Documents/3D_Gaze/dataset/PupilInvisible/office1/data_1"
elif platform == "win32":
# Windows...
    data_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\3D_Scanner_App\Apriltag1_dataset1"
    data_pi_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\PupilInvisible\office1\data_1"

# Getting the Visualization
VISUALIZATION = False
TAG_POSE_VISUALIZATION = False
DATA = "IPHONE" # "PI" or "IPHONE"

Evaluation = {}

at_detector = Detector(
        families="tagStandard41h12",
        nthreads=1,
        quad_decimate=1.0,
        quad_sigma=0.0,
        refine_edges=1,
        decode_sharpening=0.25,
        debug=0,
    )

if DATA == "IPHONE":

    images_path = os.path.join(data_path, "frames")
    pose_path = os.path.join(data_path, "pose")
    mesh_fullpath = os.path.join(data_path, "data3d/export.obj")
    depth_path = os.path.join(data_path, "depth")

elif DATA == "PI":
    data_path = data_pi_path
    images_path = os.path.join(data_path, "images_undistorted")

tag_sizes = np.arange(0.05, 0.11, 0.01)
real_tag_size = 0.087


images_files = os.listdir(images_path)

images_files.sort()

tag_points_3d = []

Vis_frames = []
if DATA == "IPHONE":

    img_width = 1440
    img_height = 1920

elif DATA == "PI":
    img_width = 1088
    img_height = 1080
    print("There is no extrinsic matrix and 3d model for data recorded by PI, so there is no 3d visualization, only visulization with tag pose")

for i, image_file in enumerate(images_files):
    # if i%5 != 0:
    #     continue
    

    img = cv2.imread(os.path.join(images_path, image_file), cv2.IMREAD_GRAYSCALE)
    color_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    if DATA == "IPHONE":
    
        camera_param_file = image_file.replace(".jpg", ".json")
        with open(os.path.join(pose_path, camera_param_file), 'r') as f:
            camera_param = json.load(f)

        intrinsics = np.array(camera_param["intrinsics"]).reshape(3, 3)

        projectionMatrix = np.array(camera_param["projectionMatrix"]).reshape(4, 4)

    elif DATA == "PI":
        intrinsics = np.array([[766.2927454396544, 0.0, 543.6272327745995],
                            [0.0, 766.3976103393867, 566.0580149497666],
                            [0.0, 0.0, 1.0]])

        projectionMatrix = np.eye(4)
        
    fxfycxcy= [intrinsics[0, 0], intrinsics[1, 1], intrinsics[0, 2], intrinsics[1, 2]]
    principal_point = np.array([intrinsics[0, 2], intrinsics[1, 2]])
    focal_length = (intrinsics[0, 0] + intrinsics[1, 1]) / 2

    # The real size of the tag is about 8.7 cm
    tags_real = at_detector.detect(img, estimate_tag_pose=True, camera_params = fxfycxcy, tag_size=real_tag_size)
    if len(tags_real) == 0:
        continue
    
    alpha_real = []
    for tag_real in tags_real:
        tag_id = tag_real.tag_id
        if tag_id not in Evaluation.keys():
            Evaluation[tag_id] = {"Real_Distance": [],
                                    "Frame_id": [],
                                    "Error_Stability": [],
                                    "Error_Accuracy": [],
                                    "angle_x": [],
                                    "angle_y": [],
                                    "angle_z": []}

            # Add the real distance
        # Project the line onto the plane which is parallel to the image plane

        R_tag2cam = np.array(tag_real.pose_R)
        t_tag2cam = np.array(tag_real.pose_t)

        r = R.from_matrix(R_tag2cam)
        euler_angles = r.as_euler('xyz', degrees=True)

        Evaluation[tag_id]["Real_Distance"].append(np.linalg.norm(t_tag2cam))
        Evaluation[tag_id]["Frame_id"].append(image_file)
        Evaluation[tag_id]["angle_x"].append(euler_angles[0])
        Evaluation[tag_id]["angle_y"].append(euler_angles[1])
        Evaluation[tag_id]["angle_z"].append(euler_angles[2])

        alpha_real_tag_mean = 0
        for i, tag_corner in enumerate(tag_real.corners):
            # 0: right bottom, 1: right top, 2: left top, 3: left bottom
            tag_center = np.array(tag_real.center)
            if i == 0:
                tag_corner_tag_coord = np.array([[1], [1], [0]]) * (real_tag_size/2)
            elif i == 1:
                tag_corner_tag_coord = np.array([[1], [-1], [0]]) * (real_tag_size/2)
            elif i == 2:
                tag_corner_tag_coord = np.array([[-1], [-1], [0]]) * (real_tag_size/2)
            elif i == 3:
                tag_corner_tag_coord = np.array([[-1], [1], [0]]) * (real_tag_size/2)
            tag_corner_cam_coord = R_tag2cam @ tag_corner_tag_coord + t_tag2cam
            tag_center_cam_coord = t_tag2cam

            delta_l = math.sqrt(math.pow((tag_corner_cam_coord[0]*tag_center_cam_coord[2]/tag_corner_cam_coord[2] - tag_center_cam_coord[0]), 2) +
                                math.pow((tag_corner_cam_coord[1]*tag_center_cam_coord[2]/tag_corner_cam_coord[2] - tag_center_cam_coord[1]), 2))

            
            delta_d = math.sqrt(math.pow(focal_length, 2) + math.pow(np.linalg.norm(tag_center - principal_point), 2))
            delta_uv_tag = np.linalg.norm(tag_corner-tag_center)
            alpha_real_tag_mean += (delta_l / real_tag_size) * (delta_d / delta_uv_tag)
        alpha_real.append(alpha_real_tag_mean / 4)

        
    
    true_distances = []
    for tag_size in tag_sizes:   
        
        tags = at_detector.detect(img, estimate_tag_pose=True, camera_params = fxfycxcy, tag_size=tag_size)
        
        # Test with only one tag
        for tag_test in tags:
            
            tag_position = np.array(tag_test.pose_t)
            tag_cam_distance = np.linalg.norm(tag_position)
            true_distances.append(tag_cam_distance)

    true_distances = np.array(true_distances).reshape(-1, len(tags))
    print(true_distances.shape)
    for i, tag in enumerate(tags_real):
        alpha_test, _, err = linear_fit(tag_sizes, true_distances[:,i])
        alpha_error = abs(alpha_real[i] - alpha_test)
        


        Evaluation[tag.tag_id]["Error_Stability"].append(err)
        Evaluation[tag.tag_id]["Error_Accuracy"].append(alpha_error)

(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 1)
(6, 2)
(6, 2)
(6, 2)


In [41]:
# if platform == "linux" or platform == "linux2":  
# # linux
#     data_path  = "/home/biyang/Documents/3D_Gaze/dataset/3D_scanner_app/Apriltag1_dataset1"
#     data_pi_path = "/home/biyang/Documents/3D_Gaze/dataset/PupilInvisible/office1/data_1"
#     evaluation_json_path = r"/home/biyang/Documents/3D_Gaze/dataset/PupilInvisible/evaluation_apriltag_detection_Iphone.json"
# elif platform == "win32":
# # Windows...
#     data_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\3D_Scanner_App\Apriltag1_dataset1"
#     data_pi_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\PupilInvisible\office1\data_1"
#     evaluation_json_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\evaluation_apriltag_detection_Iphone_a1d1.json"

# with open(evaluation_json_path, "r") as f:
#     evaluation  = json.load(f)


evaluation = Evaluation
r = R.from_euler('xyz', [0, 180, -90], degrees=True)
Additional_Rotation = r.as_matrix()
additional_rotation = np.concatenate(
                     [np.concatenate([Additional_Rotation, np.array([[0], [0], [0]])], axis=1), np.array([[0, 0, 0, 1]])], axis=0)

# Select one frame from iphone for each tag as register
track_frame = {}
for tag_id in evaluation.keys():
    distance_error = [alpha_error * 0.087 for alpha_error in evaluation[tag_id]["Error_Accuracy"]]
    distance_error = np.array(distance_error)

    index = np.argmin(distance_error)
    frame_id = evaluation[tag_id]["Frame_id"][index]

    track_frame[tag_id] = frame_id

iphone_frames_folder= os.path.join(data_path, "frames")
iphone_pose_folder = os.path.join(data_path, "pose")
mesh_fullpath = os.path.join(data_path, "data3d/textured_output.obj")

# Extract the camera pose and tag pose of each register
register = {}

for tag_id, frame_id in track_frame.items():

    camera_param_file = frame_id.replace(".jpg", ".json")
    img = cv2.imread(os.path.join(iphone_frames_folder, frame_id), cv2.IMREAD_GRAYSCALE)

    img_height, img_width = img.shape
    with open(os.path.join(iphone_pose_folder, camera_param_file), 'r') as f:
        camera_param = json.load(f)

    intrinsics = np.array(camera_param["intrinsics"]).reshape(3, 3)
    Cam2World = np.array(camera_param["cameraPoseARFrame"]).reshape(4, 4)

    fxfycxcy= [intrinsics[0, 0], intrinsics[1, 1], intrinsics[0, 2], intrinsics[1, 2]]
    tags = at_detector.detect(img, estimate_tag_pose=True, camera_params = fxfycxcy, tag_size=0.087)

    for tag in tags:
        if tag.tag_id == int(tag_id):
            tag_pose = np.concatenate(
                    [np.concatenate([np.array(tag.pose_R), np.array(tag.pose_t)], axis=1), np.array([[0, 0, 0, 1]])], axis=0)
            register[tag.tag_id] = Register(CameraPose=Cam2World, Intrinsic=intrinsics, TagPose=tag_pose)


# Compute the Registration 
# 1. Detect the tag in the frame
# 2. Find the register with common tag
# 3. Transform the new frame to the world coordinate

PI_registration = {}
PI_images_folder = os.path.join(data_pi_path, "images_undistorted")
PI_intrinsics = np.array([[766.2927454396544, 0.0, 543.6272327745995],
                            [0.0, 766.3976103393867, 566.0580149497666],
                            [0.0, 0.0, 1.0]])
PI_fxfycxcy= [PI_intrinsics[0, 0], PI_intrinsics[1, 1], PI_intrinsics[0, 2], PI_intrinsics[1, 2]]
images = os.listdir(PI_images_folder)
images.sort()

for i, img_id in enumerate(images):
    if i % 5 != 0:
        continue

    img = cv2.imread(os.path.join(PI_images_folder, img_id), cv2.IMREAD_GRAYSCALE)

    PI_tags = at_detector.detect(img, estimate_tag_pose=True, camera_params = PI_fxfycxcy, tag_size=0.087)
    if len(PI_tags) == 0:
        continue

    # Here I select simply one tag for the test
    
    PI_tag = PI_tags[0]
    Link_Register = register[PI_tag.tag_id]
    PI_tag_pose = np.concatenate(
                    [np.concatenate([np.array(PI_tag.pose_R), np.array(PI_tag.pose_t)], axis=1), np.array([[0, 0, 0, 1]])], axis=0)
    # PI_Cam2World
    PI_registration[int(img_id.split(".")[0])] =  Link_Register.CameraPose \
                                                @ additional_rotation @ Link_Register.TagPose @ np.linalg.inv(PI_tag_pose)


print(len(PI_registration.keys()))


150


In [34]:
IPhone_Registration = {}
GT_CameraPose = {}

for frame in os.listdir(iphone_frames_folder):
    if frame in track_frame.values():
        print("Passed")
        continue
    camera_param_file = frame.replace(".jpg", ".json")
    img = cv2.imread(os.path.join(iphone_frames_folder, frame), cv2.IMREAD_GRAYSCALE)

    img_height, img_width = img.shape
    with open(os.path.join(iphone_pose_folder, camera_param_file), 'r') as f:
        camera_param = json.load(f)

    intrinsics = np.array(camera_param["intrinsics"]).reshape(3, 3)
    Cam2World = np.array(camera_param["cameraPoseARFrame"]).reshape(4, 4)

    fxfycxcy= [intrinsics[0, 0], intrinsics[1, 1], intrinsics[0, 2], intrinsics[1, 2]]
    Iphone_tags = at_detector.detect(img, estimate_tag_pose=True, camera_params = fxfycxcy, tag_size=0.087)
    
    if len(Iphone_tags) == 0:
        continue
    Iphone_tag = Iphone_tags[0]
    
    
    Link_Register = register[Iphone_tag.tag_id]
    Iphone_tag_pose = np.concatenate(
                    [np.concatenate([np.array(Iphone_tag.pose_R), np.array(Iphone_tag.pose_t)], axis=1), np.array([[0, 0, 0, 1]])], axis=0)
    # PI_Cam2World
    IPhone_Registration[int(frame[6:11])] =  Link_Register.CameraPose @ additional_rotation \
                                                @ Link_Register.TagPose @ np.linalg.inv(Iphone_tag_pose)
    
    GT_CameraPose[int(frame[6:11])] = Cam2World @ additional_rotation


Passed
Passed
Passed
Passed
Passed
Passed


In [35]:
pos_error = []
rot_error = []
for gt_pose, est_pose in zip(GT_CameraPose.values(), IPhone_Registration.values()):
    pos_error.append(position_error(est_pose[:3, 3], gt_pose[:3, 3]))
    rot_error.append(rotation_error(est_pose[:3, :3], gt_pose[:3, :3]))


In [36]:
print(pos_error)
print(len(pos_error))
print(sum(pos_error)/len(pos_error))

[0.4083566926725213, 0.4019291218919338, 0.38849037585498103, 0.3179002427648748, 0.17195424301140128, 0.11458310610614114, 0.14301039197887933, 0.022714978658934494, 0.6297980737814189, 0.6552598802543246, 0.7307978960767659, 0.8091163774747812, 0.8676330072610149, 0.8760745025839376, 0.9575329461968345, 0.9874435095169811, 1.0452871353042466, 1.0755181084883743, 1.1233902567194112, 1.1597960564759109, 0.060254736393963046, 0.0716734155875048, 0.12122319913206606, 0.1498385274470514, 0.16208966344672893, 0.18995614177565434, 0.19998547469113256, 0.20044754530162393, 0.23646957514635358, 2.6106487057365686, 2.619049988957643, 2.6365642349417837, 0.7218254377285517, 0.6183800254919476, 0.39066947748013303, 1.9307558123797426, 0.6714375587603891, 1.709616233650727]
38
0.741775596240085


In [37]:
print(rot_error)

print(sum(rot_error)/len(rot_error))

[7.36927478990779, 7.187212157400475, 7.115298311392832, 6.261001362692682, 3.9361325580072655, 1.9489432525297643, 3.2921470266853063, 0.9753621536041962, 10.96642390395527, 10.455888527461664, 8.952275416433888, 7.628594003444916, 7.443266153672563, 8.12870438897756, 7.440157897565868, 7.729102133388627, 8.647531289409653, 7.000740773123268, 8.05635237001087, 9.879733784407085, 1.3850683352572002, 3.1138184356959884, 4.581211794640418, 4.727277382194506, 4.557758585460071, 5.460402817512808, 6.205918044248811, 6.7300659156342375, 7.04010462180643, 90.00804364895187, 90.03090195805373, 90.07239123681987, 11.47318425571363, 11.34330534303244, 11.385697549881717, 13.840148057493119, 7.4777019939761376, 19.630427428189975]
13.933620254174592


In [38]:
mesh_textured = o3d.io.read_triangle_mesh(mesh_fullpath)

In [42]:
visualization_registration(PI_registration, mesh_textured)

d:\Users\ybMas\anaconda3\envs\apriltag\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
d:\Users\ybMas\anaconda3\envs\apriltag\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [39]:
# Visualization of Camera Pose
#visualization_registration(PI_registration, mesh_textured)
visualization_EST_GT_IPhone(GT_CameraPose, IPhone_Registration, mesh_textured)

Output()

In [20]:
pcd_sample = mesh_textured.sample_points_uniformly(number_of_points=10000)
o3d.visualization.draw_geometries([pcd_sample])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
